## 데이터 전처리

In [1]:
import pandas as pd

In [9]:
kospi50_daily_stock_chart = pd.read_csv("./data/kospi50-1980-2021.csv")
kospi50_daily_stock_chart

,code,name,date,open,high,low,close
0,U182,kospi50_index,20000302,804.739990,833.580017,786.229980,826.950012
1,U182,kospi50_index,20000303,843.460022,845.299988,816.580017,832.000000
2,U182,kospi50_index,20000306,844.330017,849.239990,831.119995,836.200012
3,U182,kospi50_index,20000307,831.460022,833.369995,816.719971,833.349976
4,U182,kospi50_index,20000308,823.349976,847.530029,810.869995,844.239990
...,...,...,...,...,...,...,...
276194,A005380,현대차,20210831,210000.000000,212500.000000,208000.000000,212500.000000
276195,A005380,현대차,20210901,214000.000000,218500.000000,213500.000000,215000.000000
276196,A005380,현대차,20210902,214000.000000,215000.000000,210000.000000,211500.000000
276197,A005380,현대차,20210903,213000.000000,215500.000000,212000.000000,214500.000000


In [10]:
def preprocess(stock_chart):
    z_feature = stock_chart[["code", "name", "date"]].values.tolist()
    stock_chart = stock_chart.values.tolist()
    stock_first_index = 0
    
    for idx, daily in enumerate(stock_chart):
        z_feature[idx].append(daily[3] / daily[6] - 1)    # open
        z_feature[idx].append(daily[4] / daily[6] - 1)    # high
        z_feature[idx].append(daily[5] / daily[6] - 1)    # low
        
        if stock_chart[idx-1][1] != daily[1]:
            s_idx = idx
            for i in range(7):
                z_feature[idx].append(0) # close
            continue
        else:
            z_feature[idx].append(daily[6] / stock_chart[idx-1][6] - 1) # close
        
        for k in [5, 10, 15, 20, 25, 30]: # z_dk
            if (idx-s_idx)-(k-1) < 0:
                sum_close = stock_chart[s_idx:idx+1]
            else:
                sum_close = stock_chart[idx-(k-1):idx+1]

            sum_close = [chart[6] for chart in sum_close]
            sum_close = sum(sum_close) / (len(sum_close) * daily[6]) - 1
            z_feature[idx].append(sum_close)

    return pd.DataFrame(z_feature, columns=["code", "name", "date", "open", "high", "low", "close",
                                            "z_d5", "z_d10", "z_d15", "z_d20", "z_d25", "z_d30"])

In [13]:
preprocessed_kospi50_daily_stock_chart = preprocess(kospi50_daily_stock_chart)
preprocessed_kospi50_daily_stock_chart

,code,name,date,open,high,low,close,z_d5,z_d10,z_d15,z_d20,z_d25,z_d30
0,U182,kospi50_index,20000302,-0.026858,0.008017,-0.049241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,U182,kospi50_index,20000303,0.013774,0.015986,-0.018534,0.006107,-0.003035,-0.003035,-0.003035,-0.003035,-0.003035,-0.003035
2,U182,kospi50_index,20000306,0.009723,0.015594,-0.006075,0.005048,-0.005362,-0.005362,-0.005362,-0.005362,-0.005362,-0.005362
3,U182,kospi50_index,20000307,-0.002268,0.000024,-0.019956,-0.003408,-0.001470,-0.001470,-0.001470,-0.001470,-0.001470,-0.001470
4,U182,kospi50_index,20000308,-0.024744,0.003897,-0.039527,0.013068,-0.011480,-0.011480,-0.011480,-0.011480,-0.011480,-0.011480
...,...,...,...,...,...,...,...,...,...,...,...,...,...
276194,A005380,현대차,20210831,-0.011765,0.000000,-0.021176,0.011905,-0.011294,-0.019059,-0.005176,0.008471,0.015247,0.023843
276195,A005380,현대차,20210901,-0.004651,0.016279,-0.006977,0.011765,-0.017674,-0.029302,-0.017984,-0.005233,0.001581,0.010078
276196,A005380,현대차,20210902,0.011820,0.016548,-0.007092,-0.016279,0.000000,-0.010875,-0.003940,0.008038,0.016076,0.024192
276197,A005380,현대차,20210903,-0.006993,0.004662,-0.011655,0.014184,-0.008392,-0.018648,-0.019270,-0.008042,0.000466,0.007692


In [14]:
preprocessed_kospi50_daily_stock_chart.to_csv("./data/preprocessed_kospi50_daily_stock_chart.csv", index=False)

### KOSPI 31개 종목 + 지수 데이터

In [2]:
kospi31_daily_stock_chart = pd.read_csv("./data/kospi31-2003-2021.csv")
kospi31_daily_stock_chart

,code,name,date,open,high,low,close
0,U182,kospi50_index,20030102,594.500000,597.070007,584.559998,594.840027
1,U182,kospi50_index,20030103,611.070007,622.500000,608.710022,621.150024
2,U182,kospi50_index,20030106,624.799988,628.260010,618.059998,626.940002
3,U182,kospi50_index,20030107,639.080017,639.419983,612.830017,613.109985
4,U182,kospi50_index,20030108,617.380005,621.359985,611.400024,612.520020
...,...,...,...,...,...,...,...
147835,A005380,현대차,20210826,209500.000000,212000.000000,208000.000000,210000.000000
147836,A005380,현대차,20210827,209000.000000,209500.000000,205000.000000,208500.000000
147837,A005380,현대차,20210830,211000.000000,212500.000000,210000.000000,210000.000000
147838,A005380,현대차,20210831,210000.000000,212500.000000,208000.000000,212500.000000


In [20]:
preprocessed_kospi31_daily_stock_chart = preprocess(kospi31_daily_stock_chart)
preprocessed_kospi31_daily_stock_chart

,code,name,date,open,high,low,close,z_d5,z_d10,z_d15,z_d20,z_d25,z_d30
0,U182,kospi50_index,20030102,-0.000572,0.003749,-0.017282,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,U182,kospi50_index,20030103,-0.016228,0.002173,-0.020027,0.044230,-0.021178,-0.021178,-0.021178,-0.021178,-0.021178,-0.021178
2,U182,kospi50_index,20030106,-0.003413,0.002105,-0.014164,0.009321,-0.020145,-0.020145,-0.020145,-0.020145,-0.020145,-0.020145
3,U182,kospi50_index,20030107,0.042358,0.042912,-0.000457,-0.022060,0.001468,0.001468,0.001468,0.001468,0.001468,0.001468
4,U182,kospi50_index,20030108,0.007934,0.014432,-0.001829,-0.000962,0.001946,0.001946,0.001946,0.001946,0.001946,0.001946
...,...,...,...,...,...,...,...,...,...,...,...,...,...
147835,A005380,현대차,20210826,-0.002381,0.009524,-0.009524,0.002387,-0.014762,0.001190,0.017937,0.027381,0.036381,0.045238
147836,A005380,현대차,20210827,0.002398,0.004796,-0.016787,-0.007143,-0.000959,0.003357,0.020624,0.031535,0.040000,0.049081
147837,A005380,현대차,20210830,0.004762,0.011905,0.000000,0.007194,-0.002381,-0.007143,0.009365,0.022262,0.029619,0.038413
147838,A005380,현대차,20210831,-0.011765,0.000000,-0.021176,0.011905,-0.011294,-0.019059,-0.005176,0.008471,0.015247,0.023843


In [21]:
preprocessed_kospi31_daily_stock_chart.to_csv("./data/preprocessed_kospi31_daily_stock_chart.csv", index=False)

In [3]:
preprocessed_kospi31_daily_stock_chart = pd.read_csv("./data/preprocessed_kospi31_daily_stock_chart.csv")
preprocessed_kospi31_daily_stock_chart

,code,name,date,open,high,low,close,z_d5,z_d10,z_d15,z_d20,z_d25,z_d30
0,U182,kospi50_index,20030102,-0.000572,0.003749,-0.017282,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,U182,kospi50_index,20030103,-0.016228,0.002173,-0.020027,0.044230,-0.021178,-0.021178,-0.021178,-0.021178,-0.021178,-0.021178
2,U182,kospi50_index,20030106,-0.003413,0.002105,-0.014164,0.009321,-0.020145,-0.020145,-0.020145,-0.020145,-0.020145,-0.020145
3,U182,kospi50_index,20030107,0.042358,0.042912,-0.000457,-0.022060,0.001468,0.001468,0.001468,0.001468,0.001468,0.001468
4,U182,kospi50_index,20030108,0.007934,0.014432,-0.001829,-0.000962,0.001946,0.001946,0.001946,0.001946,0.001946,0.001946
...,...,...,...,...,...,...,...,...,...,...,...,...,...
147835,A005380,현대차,20210826,-0.002381,0.009524,-0.009524,0.002387,-0.014762,0.001190,0.017937,0.027381,0.036381,0.045238
147836,A005380,현대차,20210827,0.002398,0.004796,-0.016787,-0.007143,-0.000959,0.003357,0.020624,0.031535,0.040000,0.049081
147837,A005380,현대차,20210830,0.004762,0.011905,0.000000,0.007194,-0.002381,-0.007143,0.009365,0.022262,0.029619,0.038413
147838,A005380,현대차,20210831,-0.011765,0.000000,-0.021176,0.011905,-0.011294,-0.019059,-0.005176,0.008471,0.015247,0.023843


In [4]:
preprocessed_kospi31_daily_stock_chart.columns

Index(['code', 'name', 'date', 'open', 'high', 'low', 'close', 'z_d5', 'z_d10',
       'z_d15', 'z_d20', 'z_d25', 'z_d30'],
      dtype='object')